In [6]:
# ============================================================
# DAILY — Prophet Univariate + Rolling Backtest Metrics
# - Uses DAILY input (ACC_DAY)
# - Training window: last 60 months ending at cutoff date (inclusive)
# - Rolling backtest: cutoffs from first valid to last valid
#     * Each cutoff forecasts next 18 months (daily) and evaluates WAPE & Bias
# - Output includes:
#     1) backtest_detail_daily.csv  (WAPE/Bias per cutoff + flags last cutoff)
#     2) backtest_daily_org.csv     (daily actual vs yhat per cutoff)
#     3) forecast_daily_18m_org.csv (deployment forecast to FY end 2026-27)
#     4) forecast_fy_totals_org.csv (FY totals from deployment forecast)
# ============================================================

import os
import warnings
import numpy as np
import pandas as pd

# -----------------------------
# Config
# -----------------------------
INPUT_PATH = r"\\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method/0_data_input_monthly_DAILY.csv"  # <- update if needed
OUTPUT_DIR = r"\\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily_UnivarProphet"  # <- update if needed
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Daily columns (from your file)
DATE_COL = "ACC_DAY"
Y_COL = "TOT_PAID"
BUD_COL = "BUDGET_AMT"
CASE_COL = "NBR_CASE"

# Monthly-equivalent windows
TRAIN_WINDOW_MONTHS = 60
HORIZON_MONTHS = 18

# Backtest cutoff constraints
MIN_CUTOFF = pd.Timestamp("2020-03-01")  # same spirit as monthly
# Deployment forecast target: end of FY2026-27 = Mar 31, 2027
DEPLOY_FORECAST_END = pd.Timestamp("2027-03-31")

# Prophet hyperparameters (same as your single-trend)
CP = 0.05
SP = 10.0
N_CHANGEPOINTS = 25
CHANGEPOINT_RANGE = 0.95

# Outputs
OUT_BACKTEST_DETAIL = os.path.join(OUTPUT_DIR, "backtest_detail_daily.csv")
OUT_BACKTEST_DAILY  = os.path.join(OUTPUT_DIR, "backtest_daily_org.csv")
OUT_FORECAST_DAILY  = os.path.join(OUTPUT_DIR, "forecast_daily_18m_org.csv")
OUT_FY_TOTALS       = os.path.join(OUTPUT_DIR, "forecast_fy_totals_org.csv")

# -----------------------------
# Optional dependency
# -----------------------------
PROPHET_AVAILABLE = False
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception:
    PROPHET_AVAILABLE = False

if not PROPHET_AVAILABLE:
    raise RuntimeError("Prophet not available. Install with: pip install prophet")

# -----------------------------
# Helpers
# -----------------------------
def ensure_dt(s):
    return pd.to_datetime(s, errors="coerce")

def safe_to_csv(df_: pd.DataFrame, path: str, **kwargs) -> str:
    parent = os.path.dirname(path)
    if parent:
        os.makedirs(parent, exist_ok=True)
    df_.to_csv(path, **kwargs)
    return path

def fy_start_year(dt: pd.Timestamp) -> int:
    dt = pd.Timestamp(dt)
    return dt.year if dt.month >= 4 else dt.year - 1

def fiscal_year_label(dt: pd.Timestamp) -> str:
    y = fy_start_year(pd.Timestamp(dt))
    return f"FY{y}-{str(y+1)[-2:]}"

def wape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if m.sum() == 0:
        return np.nan
    denom = np.nansum(np.abs(y_true[m]))
    if denom == 0 or np.isnan(denom):
        return np.nan
    return float(100.0 * (np.nansum(np.abs(y_pred[m] - y_true[m])) / denom))

def bias_wape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if m.sum() == 0:
        return np.nan
    denom = np.nansum(np.abs(y_true[m]))
    if denom == 0 or np.isnan(denom):
        return np.nan
    return float(100.0 * (np.nansum(y_pred[m] - y_true[m]) / denom))

def prophet_univar_forecast(train_series: pd.Series, future_index: pd.DatetimeIndex):
    s = train_series.astype(float).dropna()
    if s.empty:
        return pd.Series(np.nan, index=future_index, dtype=float)

    train_df = s.reset_index()
    # Ensure Prophet sees date column as "ds"
    train_df.columns = ["ds", "y"]

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="additive",
        changepoint_prior_scale=float(CP),
        seasonality_prior_scale=float(SP),
        n_changepoints=int(N_CHANGEPOINTS),
        changepoint_range=float(CHANGEPOINT_RANGE),
    )
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        m.fit(train_df)

    fut = pd.DataFrame({"ds": future_index})
    pred = m.predict(fut)["yhat"].values
    return pd.Series(pred, index=future_index, dtype=float)

# -----------------------------
# Read & prep daily data
# -----------------------------
df = pd.read_csv(INPUT_PATH)
df.columns = [c.strip() for c in df.columns]

required = [DATE_COL, Y_COL, BUD_COL, CASE_COL]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df[DATE_COL] = ensure_dt(df[DATE_COL])
df = df.dropna(subset=[DATE_COL]).copy()

for c in [Y_COL, BUD_COL, CASE_COL]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.sort_values(DATE_COL).set_index(DATE_COL)

# Enforce daily grid (keeps gaps as NaN; evaluation will ignore NaNs)
df = df.asfreq("D")

# Last actual day for y (use max day where y is non-null)
y_non_null = df[Y_COL].dropna()
if y_non_null.empty:
    raise ValueError("No non-null target values found in TOT_PAID.")
last_actual_day = pd.Timestamp(y_non_null.index.max())

print("Last actual day (TOT_PAID):", last_actual_day.date())

# -----------------------------
# Build rolling cutoffs
# Each cutoff must allow a full 18-month test window of actuals.
# Last cutoff corresponds to the last 18 months of available actual data.
# -----------------------------
test_span = pd.DateOffset(months=HORIZON_MONTHS)
train_span = pd.DateOffset(months=TRAIN_WINDOW_MONTHS)

last_cutoff = last_actual_day - test_span
all_days = df.index

cutoffs = []
for d in all_days:
    if d < MIN_CUTOFF:
        continue
    if d > last_cutoff:
        break
    # Require at least some training history
    if d - train_span >= all_days.min():
        cutoffs.append(pd.Timestamp(d))

if not cutoffs:
    raise ValueError("No valid rolling cutoffs found. Check MIN_CUTOFF / data range.")

print(f"Rolling cutoffs: {len(cutoffs)} | first={cutoffs[0].date()} | last={cutoffs[-1].date()}")
print(f"Last cutoff test window should cover: {(cutoffs[-1] + pd.Timedelta(days=1)).date()} to {(cutoffs[-1] + test_span).date()}")

# -----------------------------
# Rolling backtest
# -----------------------------
detail_rows = []
daily_rows = []

for cutoff in cutoffs:
    train_start = cutoff - train_span + pd.Timedelta(days=1)
    train_end = cutoff
    test_start = cutoff + pd.Timedelta(days=1)
    test_end = cutoff + test_span

    train = df.loc[train_start:train_end].copy()
    test = df.loc[test_start:test_end].copy()

    # Need enough non-null train + test y
    y_train = train[Y_COL].dropna().astype(float)
    y_test  = test[Y_COL].astype(float)  # keep NaNs; eval filters them out

    if y_train.empty:
        continue
    if test.index.min() is pd.NaT or test.index.max() is pd.NaT:
        continue

    # Forecast daily for full test index
    test_index = pd.date_range(test_start, test_end, freq="D")
    try:
        pred = prophet_univar_forecast(y_train, test_index)
        err = None
        ok = True
    except Exception as e:
        pred = pd.Series(np.nan, index=test_index, dtype=float)
        err = e
        ok = False

    # Build daily rows (actual + yhat) for this cutoff
    actual_slice = df.loc[test_start:test_end, [Y_COL, BUD_COL, CASE_COL]].copy()
    actual_slice = actual_slice.reindex(test_index)

    out = actual_slice.join(pred.rename("yhat_org"), how="outer")
    out.index.name = "ds"
    out = out.reset_index().rename(columns={
        Y_COL: "y_actual",
        BUD_COL: "Budget_AMT",
        CASE_COL: "Nbr_Cases",
    })
    out["cutoff"] = pd.Timestamp(cutoff)
    out["model"] = "M2_Prophet_univariate_daily"
    out["FY"] = out["ds"].apply(fiscal_year_label)

    daily_rows.extend(out.to_dict("records"))

    # Evaluate WAPE/Bias on finite pairs only (your edited logic)
    eval_df = out.copy()
    eval_df["y_actual"] = pd.to_numeric(eval_df["y_actual"], errors="coerce")
    eval_df["yhat_org"] = pd.to_numeric(eval_df["yhat_org"], errors="coerce")

    eval_df = eval_df[
        np.isfinite(eval_df["y_actual"]) & np.isfinite(eval_df["yhat_org"])
    ]

    w = wape(eval_df["y_actual"].values, eval_df["yhat_org"].values) if not eval_df.empty else np.nan
    b = bias_wape(eval_df["y_actual"].values, eval_df["yhat_org"].values) if not eval_df.empty else np.nan

    detail_rows.append({
        "model": "M2_Prophet_univariate_daily",
        "cutoff": pd.Timestamp(cutoff),
        "train_start": pd.Timestamp(train_start),
        "train_end": pd.Timestamp(train_end),
        "test_start": pd.Timestamp(test_start),
        "test_end": pd.Timestamp(test_end),
        "n_train_days": int(train.loc[train_start:train_end].shape[0]),
        "n_test_days": int(test_index.shape[0]),
        "n_eval_days": int(eval_df.shape[0]),
        "WAPE_pct": float(w) if np.isfinite(w) else np.nan,
        "Bias_WAPE_pct": float(b) if np.isfinite(b) else np.nan,
        "ok": bool(ok) and np.isfinite(w),
        "error": "" if err is None else str(err)[:500],
        "is_last_cutoff": bool(cutoff == cutoffs[-1]),
    })

# Save rolling outputs
detail_df = pd.DataFrame(detail_rows).sort_values("cutoff").reset_index(drop=True)
daily_df  = pd.DataFrame(daily_rows)

safe_to_csv(detail_df, OUT_BACKTEST_DETAIL, index=False)
safe_to_csv(daily_df, OUT_BACKTEST_DAILY, index=False)

print("✅ Saved rolling backtest outputs:")
print(" -", OUT_BACKTEST_DETAIL)
print(" -", OUT_BACKTEST_DAILY)

# -----------------------------
# Explicitly show the LAST cutoff metrics (last 18 months of actual data)
# -----------------------------
last_row = detail_df[detail_df["is_last_cutoff"] == True].tail(1)
if not last_row.empty:
    r = last_row.iloc[0]
    print("\nLAST CUTOFF (last 18 months of actuals):")
    print(" cutoff:", pd.Timestamp(r["cutoff"]).date())
    print(" test window:", pd.Timestamp(r["test_start"]).date(), "to", pd.Timestamp(r["test_end"]).date())
    print(" WAPE%:", r["WAPE_pct"])
    print(" Bias%:", r["Bias_WAPE_pct"])

# -----------------------------
# Deployment forecast: fit on last 60 months, forecast to FY end (Mar 31, 2027)
# -----------------------------
deploy_train_end = last_actual_day
deploy_train_start = deploy_train_end - train_span + pd.Timedelta(days=1)

deploy_train = df.loc[deploy_train_start:deploy_train_end, Y_COL].dropna().astype(float)
if deploy_train.empty:
    raise RuntimeError("Deployment training window is empty after filtering.")

future_index = pd.date_range(deploy_train_end + pd.Timedelta(days=1), DEPLOY_FORECAST_END, freq="D")

deploy_pred = prophet_univar_forecast(deploy_train, future_index)

fc_out = pd.DataFrame({
    "ds": future_index,
    "model": "M2_Prophet_univariate_daily",
    "yhat_org": deploy_pred.values,
})
fc_out["FY"] = fc_out["ds"].apply(fiscal_year_label)

safe_to_csv(fc_out, OUT_FORECAST_DAILY, index=False)

# FY totals from deployment forecast
fy_totals = (
    fc_out.groupby("FY", as_index=False)
          .agg(yhat_org_total=("yhat_org", "sum"))
)
safe_to_csv(fy_totals, OUT_FY_TOTALS, index=False)

print("\n✅ Saved deployment outputs:")
print(" -", OUT_FORECAST_DAILY)
print(" -", OUT_FY_TOTALS)

Last actual day (TOT_PAID): 2025-12-31
Rolling cutoffs: 1583 | first=2020-03-01 | last=2024-06-30
Last cutoff test window should cover: 2024-07-01 to 2025-12-30


20:45:49 - cmdstanpy - INFO - Chain [1] start processing
20:45:49 - cmdstanpy - INFO - Chain [1] done processing
20:45:50 - cmdstanpy - INFO - Chain [1] start processing
20:45:50 - cmdstanpy - INFO - Chain [1] done processing
20:45:51 - cmdstanpy - INFO - Chain [1] start processing
20:45:51 - cmdstanpy - INFO - Chain [1] done processing
20:45:52 - cmdstanpy - INFO - Chain [1] start processing
20:45:53 - cmdstanpy - INFO - Chain [1] done processing
20:45:54 - cmdstanpy - INFO - Chain [1] start processing
20:45:54 - cmdstanpy - INFO - Chain [1] done processing
20:45:55 - cmdstanpy - INFO - Chain [1] start processing
20:45:55 - cmdstanpy - INFO - Chain [1] done processing
20:45:56 - cmdstanpy - INFO - Chain [1] start processing
20:45:56 - cmdstanpy - INFO - Chain [1] done processing
20:45:57 - cmdstanpy - INFO - Chain [1] start processing
20:45:57 - cmdstanpy - INFO - Chain [1] done processing
20:45:58 - cmdstanpy - INFO - Chain [1] start processing
20:45:58 - cmdstanpy - INFO - Chain [1]

✅ Saved rolling backtest outputs:
 - \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily_UnivarProphet\backtest_detail_daily.csv
 - \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily_UnivarProphet\backtest_daily_org.csv

LAST CUTOFF (last 18 months of actuals):
 cutoff: 2024-06-30
 test window: 2024-07-01 to 2025-12-30
 WAPE%: 75.52252225070261
 Bias%: 6.364138179020819


21:14:34 - cmdstanpy - INFO - Chain [1] start processing
21:14:34 - cmdstanpy - INFO - Chain [1] done processing



✅ Saved deployment outputs:
 - \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily_UnivarProphet\forecast_daily_18m_org.csv
 - \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily_UnivarProphet\forecast_fy_totals_org.csv
